In [5]:
# Add project root to path so we can import nanochat
import sys
sys.path.insert(0, "/Users/yanxia/code/lab/nanochat")

In [7]:
from nanochat.common import compute_init, compute_cleanup, get_dist_info, print0

compute_init("mps")
info = get_dist_info()
# print0(f"Rank: {info.rank}, World: {info.world_size}")
compute_cleanup()

2026-01-12 16:07:21,590 - nanochat.common - INFO - Distributed world size: 1


In [8]:
from nanochat.tokenizer import RustBPETokenizer

In [9]:
# Train tiny tokenizer
texts = ["Hello world!", "The quick brown fox.", "Machine learning is fun."] * 100
tok = RustBPETokenizer.train_from_iterator(iter(texts), vocab_size=512)

# Test encoding/decoding
text = "Hello machine learning world!"
ids = tok.encode(text)
print(f"Text: {text}")
print(f"Tokens: {ids}")
print(f"Decoded: {tok.decode(ids)}")

# Test conversation rendering
conv = {
    "messages": [
        {"role": "user", "content": "What is 2+2?"},
        {"role": "assistant", "content": "4"},
    ]
}
rendered = tok.render_conversation(conv)
print(f"Rendered: {rendered}")

2026-01-12 16:07:28,110 - rustbpe - INFO - Processing sequences from iterator (buffer_size: 8192)
2026-01-12 16:07:28,113 - rustbpe - INFO - Processed 300 sequences total, 12 unique
2026-01-12 16:07:28,113 - rustbpe - INFO - Starting BPE training: 247 merges to compute
2026-01-12 16:07:28,113 - rustbpe - INFO - Computing initial pair counts from 12 unique sequences
2026-01-12 16:07:28,114 - rustbpe - INFO - Building heap with 41 unique pairs
2026-01-12 16:07:28,114 - rustbpe - INFO - Starting merge loop
2026-01-12 16:07:28,115 - rustbpe - INFO - Progress: 1% (3/247 merges) - Last merge: (32, 98) -> 258 (frequency: 100)
2026-01-12 16:07:28,115 - rustbpe - INFO - Progress: 2% (5/247 merges) - Last merge: (32, 108) -> 260 (frequency: 100)
2026-01-12 16:07:28,116 - rustbpe - INFO - Progress: 3% (8/247 merges) - Last merge: (72, 101) -> 263 (frequency: 100)
2026-01-12 16:07:28,116 - rustbpe - INFO - Progress: 4% (10/247 merges) - Last merge: (84, 104) -> 265 (frequency: 100)
2026-01-12 16:0

Text: Hello machine learning world!
Tokens: [295, 32, 109, 97, 267, 282, 293, 294, 33]
Decoded: Hello machine learning world!
Rendered: ([297, 298, 87, 104, 97, 116, 284, 32, 50, 43, 50, 63, 299, 300, 52, 301], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])


In [3]:
def find_num_heads(model_dim, target_head_dim=128):
    # Find num_heads that divides model_dim evenly, with head_dim closest to target.
    ideal = max(1, round(model_dim / target_head_dim))
    for offset in range(model_dim):
        for candidate in [ideal + offset, ideal - offset]:
            if candidate > 0 and model_dim % candidate == 0:
                return candidate
    return 1

print(find_num_heads(1024))

8
